In [1]:
include("LiPoSID.jl")
using QuantumOptics
basis = NLevelBasis(2)
using DynamicPolynomials
using Plots

using JuMP
using NLopt

### Load exact data

In [2]:
γᵗˣᵗ = "0.079477"

parentdir = pwd()
data_dir = parentdir*"\\DATA\\"
println(data_dir)

ρᵍ, tᵍ = LiPoSID.get_rho_series(data_dir*"State_B1_2CUT_data.h5", γᵗˣᵗ)
ρᵉ, tᵉ = LiPoSID.get_rho_series(data_dir*"State_B2_2CUT_data.h5", γᵗˣᵗ)
ρˣ, tˣ = LiPoSID.get_rho_series(data_dir*"State_B3_2CUT_data.h5", γᵗˣᵗ)
ρʸ, tʸ = LiPoSID.get_rho_series(data_dir*"State_B4_2CUT_data.h5", γᵗˣᵗ)

@assert tᵍ == tᵉ == tˣ == tʸ 

t = convert(Vector{Float64}, tᵉ)
@assert maximum(diff(t)) ≈ minimum(diff(t)) ≈ t[2]-t[1]
Δt = t[2]-t[1]
t_steps = length(t)

C:\Users\Zakhar\Documents\GitHub\POP_fail\DATA\


1256

### Set up for POP

In [86]:
#using DynamicPolynomials

@polyvar e[1:2]
@polyvar d[1:2]

    Hˢʸᵐᵇ = [ e[1]               d[1] + im*d[2]
              d[1] - im*d[2]     0.             ]

    Hˢʸᵐᵇₐₙ = [ e[1]   0
                0      0. ]

@polyvar a[1:4]
@polyvar b[1:4]

    Aˢʸᵐᵇ = [ a[1] + im*b[1]           a[3] + im*b[3]
              a[4] + im*b[4]           a[2] + im*b[2]   ]

    Aˢʸᵐᵇₜᵣ₀ = [ a[1] + im*b[1]           a[2] + im*b[2]
                a[3] + im*b[3]           -a[1] - im*b[1]            ]

    Aˢʸᵐᵇₐₙ = [ 0      a[2] 
                0      0.  ]

obj = 0

objₐₙ = 0

objₐₙᵀʳ⁰ = 0

for ρ in [ρᵍ, ρᵉ, ρˣ, ρʸ]

    # Convert cut ρ series:
    ρ = convert(Vector{Matrix{ComplexF64}}, ρ)

    obj += LiPoSID.simpson_obj(ρ, t,  Hˢʸᵐᵇ, [Aˢʸᵐᵇ])
    
    objₐₙ += LiPoSID.simpson_obj(ρ, t,  Hˢʸᵐᵇₐₙ, [Aˢʸᵐᵇₐₙ])
    
    objₐₙᵀʳ⁰ += LiPoSID.simpson_obj(ρ, t,  Hˢʸᵐᵇₐₙ, [Aˢʸᵐᵇₜᵣ₀])

end # of files (initial states) loop 


#### Use cubic regularization

we apply regularization towards the elements of dissipator only

In [93]:
α = 0.01
objʳᵉᵍ = obj + α * sum(a.^3 + b.^3) #+ α * sum(a[1:2].^2 + b[1:2].^2);

#objʳᵉᵍ = obj + α * sum(a.^3 + b.^3);

#objʳᵉᵍᵀʳ⁰ = objₐₙᵀʳ⁰ + sum(a[3:4].^3 + b[3:4].^3);

#### Global optimization with TSSOS

In [94]:
sol, best_method = LiPoSID.sos_min_newton(obj) 

************************TSSOS************************
TSSOS is launching...
optimum = 0.0005504932393311856
The local solver failed!
No higher TSSOS hierarchy!
************************TSSOS************************
TSSOS is launching...
optimum = 0.0005503304415601116
Global optimality certified!
No higher TSSOS hierarchy!
val_p = [0.01185787884645606, 0.0005504405385750033]


(PolyVar{true}[e₁, d₁, d₂, a₁, a₂, a₃, a₄, b₁, b₂, b₃, b₄] => [25.125920882326525, 0.43431106099966577, 0.07639359345164903, -2.442056734004851, -2.4420207437783734, 0.15710476818728356, 0.0389520382837994, -0.7027266507795179, -0.7027907343513116, -0.24614633395442415, -0.05309244436755465], "scaled_tssos")

In [95]:
solₐₙ, best_methodₐₙ = LiPoSID.sos_min_newton(objₐₙ) 

************************TSSOS************************
TSSOS is launching...
optimum = 0.0013068260669271202
Global optimality certified!
No higher TSSOS hierarchy!
************************TSSOS************************
TSSOS is launching...
optimum = 0.0013068175921014087
Found a local optimal solution giving an upper bound: 0.011857047039541158 and a relative optimality gap: 0.01055022944743975.
No higher TSSOS hierarchy!
val_p = [0.0013068163059983817, 0.011857047039598001]


(PolyVar{true}[e₁, a₂] => [25.126072080100073, 0.28193614217535945], "tssos")

In [96]:
#LiPoSID.sos_min_newton(objʳᵉᵍᵀʳ⁰) 

In [97]:
solʳᵉᵍ, best_methodʳᵉᵍ = LiPoSID.sos_min_newton(objʳᵉᵍ)

************************TSSOS************************
TSSOS is launching...
termination status: SLOW_PROGRESS
solution status: UNKNOWN_RESULT_STATUS
optimum = -7.676890801142512e6
The local solver failed!
No higher TSSOS hierarchy!
************************TSSOS************************
TSSOS is launching...
termination status: SLOW_PROGRESS
solution status: UNKNOWN_RESULT_STATUS
optimum = -1.1186198281857157e7
The local solver failed!
No higher TSSOS hierarchy!
val_p = [2256.950099519192, 19403.791465854334]


(PolyVar{true}[e₁, d₁, d₂, a₁, a₂, a₃, a₄, b₁, b₂, b₃, b₄] => [-0.19739075652348204, -0.07267114307189257, -0.7286444979787724, -0.28691047304874284, 0.75548226216534, 0.8021538231267146, -3.261741668780403, 1.164798041560876, -0.9230322722839951, -0.7067571438835434, -0.5399402743837877], "tssos")

#### Overfitting

In [100]:
subs(obj, sol), subs(objₐₙ, solₐₙ), subs(objʳᵉᵍ, solʳᵉᵍ)

(0.0005504405385750033, 0.0013068163059983817, 2256.950099519192)

#### Regularization improves Hamiltonian

In [71]:
Hᵖᵒᵖ = subs(Hˢʸᵐᵇ, sol)

2×2 Matrix{Polynomial{true, ComplexF64}}:
 (25.1258+0.0im)        (0.745239+0.209842im)
 (0.745239-0.209842im)  0.0+0.0im

In [72]:
Hᵖᵒᵖᵃⁿᶻ = subs(Hˢʸᵐᵇₐₙ, solₐₙ)

2×2 Matrix{Term{true, Float64}}:
 25.1261  0.0
 0.0      0.0

In [73]:
subs(Hˢʸᵐᵇ, solʳᵉᵍ)

2×2 Matrix{Polynomial{true, ComplexF64}}:
 (25.1261+0.0im)           (4.2229e-5-3.45035e-5im)
 (4.2229e-5+3.45035e-5im)  0.0+0.0im

#### Regularization slightly improves dissipator

In [74]:
Aᵖᵒᵖ = subs(Aˢʸᵐᵇ, sol)

2×2 Matrix{Polynomial{true, ComplexF64}}:
 (-3.81609+2.59401im)       (-0.0478286-0.288067im)
 (-0.00655189-0.0655216im)  (-3.81611+2.59394im)

In [75]:
Aᵖᵒᵖᵃⁿᶻ = subs(Aˢʸᵐᵇₐₙ, solₐₙ)

2×2 Matrix{Term{true, Float64}}:
 0.0  0.281936
 0.0  0.0

In [76]:
subs(Aˢʸᵐᵇ, solʳᵉᵍ)

2×2 Matrix{Polynomial{true, ComplexF64}}:
 (1.91632e-5+9.22896e-6im)  (-0.318609+0.000276416im)
 (-0.0795867-0.00471216im)  (-1.91632e-5-9.22896e-6im)

#### Local minimization with our regularization

In [16]:
function local_min(obj, guess)  
    
    vars = variables(obj)
    
    function g(a...)
    # Converting polynomial expression of objective to function to be minimized
    obj(vars => a)
    end

    # Create NLopt model
    model = Model(NLopt.Optimizer)

    # Set algorithm 
    set_optimizer_attribute(model, "algorithm", :LD_SLSQP) 

    # Set variables
    @variable(model, y[1:length(vars)]);

    # Register objective
    register(model, :g, length(y), g; autodiff = true)
    @NLobjective(model, Min, g(y...))

    # Set guess

    for (var, init_val) in zip(y, guess)
        set_start_value(var, init_val)
    end

    # Call JuMP optimization function
    JuMP.optimize!(model)

    solution = vars => map(value, y)

    return solution

end

local_min (generic function with 1 method)

In [17]:
solˡᵒᶜ = local_min(obj, solʳᵉᵍ[2])

PolyVar{true}[e₁, d₁, d₂, a₁, a₂, a₃, a₄, b₁, b₂, b₃, b₄] => [25.126102651427257, 4.43804653365982e-5, -3.798606754027497e-5, 3.712760616141546e-5, -3.7127819570712876e-5, -0.29201003731021996, -0.06571300161680653, -2.201317833212569e-6, 2.1302654653652793e-6, 0.0002103341882541645, -0.004218409992500306]

In [33]:
Hʳᵉᵍ = subs(Hˢʸᵐᵇ, solʳᵉᵍ)

2×2 Matrix{Polynomial{true, ComplexF64}}:
 (25.1261+0.0im)            (4.42497e-5-3.77864e-5im)
 (4.42497e-5+3.77864e-5im)  0.0+0.0im

In [34]:
subs(Hˢʸᵐᵇ, solˡᵒᶜ)

2×2 Matrix{Polynomial{true, ComplexF64}}:
 (25.1261+0.0im)            (4.43805e-5-3.79861e-5im)
 (4.43805e-5+3.79861e-5im)  0.0+0.0im

In [35]:
subs(Aˢʸᵐᵇ, solʳᵉᵍ)

2×2 Matrix{Polynomial{true, ComplexF64}}:
 (3.41667e-5-7.72948e-7im)  (-0.294549+0.000207068im)
 (-0.0669974-0.00427837im)  (-3.41667e-5+7.72948e-7im)

In [36]:
Aʳᵉᵍ = subs(Aˢʸᵐᵇ, solˡᵒᶜ)

2×2 Matrix{Polynomial{true, ComplexF64}}:
 (3.71276e-5-2.20132e-6im)  (-0.29201+0.000210334im)
 (-0.065713-0.00421841im)   (-3.71278e-5+2.13027e-6im)

#### Consider fidelity of identification for the worst state

In [37]:
#Exact data
#γᵗˣᵗ = "0.079477"
df = "State_D20"
ρ, t = LiPoSID.get_rho_series(data_dir*df*"_2CUT_data.h5", string(γᵗˣᵗ))
ρ = convert(Vector{Matrix{ComplexF64}}, ρ)
bₑ = LiPoSID.bloch(ρ);
t = convert(Vector{Float64}, t);
max_step = length(t)

1256

In [38]:
Hᵃⁿᶻ = [25.126 0
        0      0 ]

Aᵃⁿᶻ = [0   √parse(Float64, γᵗˣᵗ)
        0     0 ]

ρᵃⁿᶻ = LiPoSID.Lindblad_time_evolution(basis, ρ[1], t, Hᵃⁿᶻ, [Aᵃⁿᶻ]);
bᵃⁿᶻ = LiPoSID.bloch(ρᵃⁿᶻ);

┌ Warning: Backwards compatability support of the new return codes to Symbols will be deprecated with the Julia v1.9 release. Please see https://docs.sciml.ai/SciMLBase/stable/interfaces/Solutions/#retcodes for more information
└ @ SciMLBase C:\Users\Zakhar\.julia\packages\SciMLBase\VKnrY\src\retcodes.jl:360


In [39]:
ρᵖᵒᵖ = LiPoSID.Lindblad_time_evolution(basis, ρ[1], t, Hᵖᵒᵖ, [Aᵖᵒᵖ]);
bᵖᵒᵖ = LiPoSID.bloch(ρᵖᵒᵖ);

┌ Warning: Backwards compatability support of the new return codes to Symbols will be deprecated with the Julia v1.9 release. Please see https://docs.sciml.ai/SciMLBase/stable/interfaces/Solutions/#retcodes for more information
└ @ SciMLBase C:\Users\Zakhar\.julia\packages\SciMLBase\VKnrY\src\retcodes.jl:360


In [40]:
ρᵖᵒᵖᵃⁿᶻ = LiPoSID.Lindblad_time_evolution(basis, ρ[1], t, Hᵖᵒᵖᵃⁿᶻ, [Aᵖᵒᵖᵃⁿᶻ]);
bᵖᵒᵖᵃⁿᶻ = LiPoSID.bloch(ρᵃⁿᶻ);

┌ Warning: Backwards compatability support of the new return codes to Symbols will be deprecated with the Julia v1.9 release. Please see https://docs.sciml.ai/SciMLBase/stable/interfaces/Solutions/#retcodes for more information
└ @ SciMLBase C:\Users\Zakhar\.julia\packages\SciMLBase\VKnrY\src\retcodes.jl:360


In [41]:
ρʳᵉᵍ = LiPoSID.Lindblad_time_evolution(basis, ρ[1], t, Hʳᵉᵍ, [Aʳᵉᵍ])
bʳᵉᵍ = LiPoSID.bloch(ρʳᵉᵍ);

┌ Warning: Backwards compatability support of the new return codes to Symbols will be deprecated with the Julia v1.9 release. Please see https://docs.sciml.ai/SciMLBase/stable/interfaces/Solutions/#retcodes for more information
└ @ SciMLBase C:\Users\Zakhar\.julia\packages\SciMLBase\VKnrY\src\retcodes.jl:360


In [42]:
fʳᵉᵍ = LiPoSID.fidelity_series(basis, ρ[1:max_step], ρʳᵉᵍ[1:max_step])
fᵃⁿᶻ = LiPoSID.fidelity_series(basis, ρ[1:max_step], ρᵃⁿᶻ[1:max_step]);
fᵖᵒᵖᵃⁿᶻ = LiPoSID.fidelity_series(basis, ρ[1:max_step], ρᵖᵒᵖᵃⁿᶻ[1:max_step])
fᵖᵒᵖ = LiPoSID.fidelity_series(basis, ρ[1:max_step], ρᵖᵒᵖ[1:max_step]);

In [43]:
plot(fʳᵉᵍ, label = "regularized POP")
plot!(fᵖᵒᵖ, label = "POP")
#plot!(fᵃⁿᶻ, label = "ansatz")
#plot!(fᵖᵒᵖᵃⁿᶻ, label = "POP ansatz")

### Trajectories

In [44]:
function xyz(b)
    x = b[1,:]
    y = b[2,:]
    z = b[3,:]
    return x,y,z
end

xyz (generic function with 1 method)

In [47]:
Plots.plotlyjs()
plt = Plots.plot(xyz(bₑ), label="exact")
plt = Plots.plot!(xyz(bᵃⁿᶻ), label="ansatz")
plt = Plots.plot!(xyz(bᵖᵒᵖᵃⁿᶻ), label="pop-anz")
plt = Plots.plot!(xyz(bᵖᵒᵖ), label="pop")
plt = Plots.plot!(xyz(bʳᵉᵍ), label="reg-pop")